In [93]:
import pandas as pd
import numpy as np
import scipy


%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.colors as colors

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


from sklearn.feature_extraction.text import CountVectorizer 

import string
from nltk.corpus import stopwords

import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import colorlover as cl
#py.sign_in('Xiaoye127', 'xiaoye123456789') 


# SparkSql 
 review(term-freq)         
 customer information(customer_info)

## read data

In [2]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SQLContext

In [3]:
sc = pyspark.SparkContext()
sqlContext = SQLContext(sc)


# # custom_info 
# custom_spark = sqlContext.read.csv("/Users/xy/Desktop/BIG DATA MGMT SYSTM/Project/customer_info.csv",
#                                    header=True, inferSchema=True)
# custom_spark.show(5)



# # term_freq 
# term_spark = sqlContext.read.csv("/Users/xy/Desktop/BIG DATA MGMT SYSTM/Project/term_frequency.csv",
#                                    header=True, inferSchema=True)


# # all_data
# all_spark=sqlContext.read.csv("/Users/xy/Desktop/BIG DATA MGMT SYSTM/Project/all_data.csv",
#                                    header=True, inferSchema=True)
# all_spark.show(5)



# #brand.csv
# brand_spark=sqlContext.read.csv("/Users/xy/Desktop/BIG DATA MGMT SYSTM/Project/Data/Prducts_Info.csv",
#                                    header=True, inferSchema=True)
# brand_spark.show(5)


# joined data
joined_spark=sqlContext.read.csv("/Users/xy/Desktop/BIG DATA MGMT SYSTM/Project/joined_data.csv",
                                   header=True, inferSchema=True)
joined_spark.show(2)

+---+---------+----------+-------------+------+-----+--------+---------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------+
|_c0|fileIndex|productID2| userNickname|rating|  age|eyeColor|hairColor|skinTone|skinType|              review|         ProductID11|        Product_name|            Brand_id|    Brand_name|
+---+---------+----------+-------------+------+-----+--------+---------+--------+--------+--------------------+--------------------+--------------------+--------------------+--------------+
|  0|        0|   P307803|        LCD11|     5|35-44|    null|     null|    null|    null|I'm a fan of gour...|             P307803|Blu Mediterraneo ...|                5847|Acqua Di Parma|
|  1|        1|   P307803|GabrielleShae|     5| null|    null|     null|    null|    null|"I received this ...| but it is very s...|             P307803|Blu Mediterraneo ...|          5847|
+---+---------+----------+-------------+------+---

In [4]:
# term_spark.createOrReplaceTempView('term_freq')
# custom_spark.createOrReplaceTempView('custom_info')
# all_spark.createOrReplaceTempView('all_data')
# brand_spark.createOrReplaceTempView('brand')
joined_spark.createOrReplaceTempView('joined')

## spark sql 

###  try term_freq

In [89]:
# try term_freq
term_input=sqlContext.sql('SELECT _c0 as index ,nars FROM term_freq WHERE nars!=0 ')
term_input.show(20)

+------+----+
| index|nars|
+------+----+
| 16868|   1|
| 16927|   1|
| 16980|   1|
| 23054|   1|
| 23146|   1|
|104605|   1|
|160200|   1|
|169914|   1|
|183202|   1|
|320478|   1|
|320543|   1|
|338482|   1|
|351905|   1|
|362074|   1|
|362195|   1|
|370336|   1|
|373968|   1|
|378322|   1|
|387897|   1|
|441944|   1|
+------+----+
only showing top 20 rows



### try custom_info

In [44]:
# try custom_info
custom_input=sqlContext.sql('''SELECT fileIndex ,rating 
                               FROM custom_info 
                               WHERE age=\'35-44\' and eyeColor= \'Brown\'
                               ''')
custom_input.show()


+---------+------+
|fileIndex|rating|
+---------+------+
|       34|     3|
|      194|     5|
|      359|     5|
|      442|     3|
|      520|     5|
|      633|     5|
|      785|     5|
|      796|     5|
|     1053|     1|
|     1108|     5|
|     1146|     5|
|     1178|     5|
|     1213|     5|
|     1233|     1|
|     1646|     5|
|     1679|     4|
|     2222|     5|
|     2419|     3|
|     3139|     5|
|     4094|     5|
+---------+------+
only showing top 20 rows



### try all_data

In [30]:
# try all_data
all_input=sqlContext.sql('''SELECT *
                               FROM all_data 
                               WHERE review REGEXP 'dark circle' 
                               AND age='35-44'
                               ''')
all_input.show()

+-----+---------+---------+----------------+------+-----+--------+---------+--------+-----------+--------------------+
|  _c0|fileIndex|productID|    userNickname|rating|  age|eyeColor|hairColor|skinTone|   skinType|              review|
+-----+---------+---------+----------------+------+-----+--------+---------+--------+-----------+--------------------+
| 1755|     1755|  P282938|           tvk09|     5|35-44|    null|     null|    null|Combination|Bags on top of ey...|
| 1764|     1764|  P282938|   kmharshberger|     5|35-44|    null|     null|    null|Combination|My mom, sister, a...|
| 1786|     1786|  P282938|      Leslielynn|     5|35-44|    null|     null|    null|Combination|Really rich but n...|
| 1801|     1801|  P282938|   kmharshberger|     5|35-44|    null|     null|    null|Combination|My mom, sister, a...|
| 1843|     1843|  P282938|          sheri5|     5|35-44|    null|     null|    Fair|     Normal|I had been using ...|
| 1896|     1896|  P282938|         julabee|    

### for joined table

In [327]:
# for joined table
result=sqlContext.sql('''SELECT *
                               FROM joined 
                               WHERE review REGEXP 'dark circle' 
                               AND age='35-44'
                               ''')
result.show()

+-----+---------+----------+----------------+------+-----+--------+---------+--------+-----------+--------------------+-----------+--------------------+--------+-----------------+
|  _c0|fileIndex|productID2|    userNickname|rating|  age|eyeColor|hairColor|skinTone|   skinType|              review|ProductID11|        Product_name|Brand_id|       Brand_name|
+-----+---------+----------+----------------+------+-----+--------+---------+--------+-----------+--------------------+-----------+--------------------+--------+-----------------+
| 1755|     1755|   P282938|           tvk09|     5|35-44|    null|     null|    null|Combination|Bags on top of ey...|    P282938|Complete Eye Rene...|    6018|         Algenist|
| 1764|     1764|   P282938|   kmharshberger|     5|35-44|    null|     null|    null|Combination|My mom, sister, a...|    P282938|Complete Eye Rene...|    6018|         Algenist|
| 1786|     1786|   P282938|      Leslielynn|     5|35-44|    null|     null|    null|Combination|Re

# for visulization

In [328]:
result_df = result.toPandas()
result_df['rating']=result_df['rating'].astype(int)
result_df.head()

,_c0,fileIndex,productID2,userNickname,rating,age,eyeColor,hairColor,skinTone,skinType,review,ProductID11,Product_name,Brand_id,Brand_name
0,1755,1755,P282938,tvk09,5,35-44,None,None,None,Combination,"Bags on top of eyelids are noticeably tighter,...",P282938,Complete Eye Renewal Balm,6018,Algenist
1,1764,1764,P282938,kmharshberger,5,35-44,None,None,None,Combination,"My mom, sister, and grandma all have dark circ...",P282938,Complete Eye Renewal Balm,6018,Algenist
2,1786,1786,P282938,Leslielynn,5,35-44,None,None,None,Combination,Really rich but not cloggy feeling. Smooth and...,P282938,Complete Eye Renewal Balm,6018,Algenist
3,1801,1801,P282938,kmharshberger,5,35-44,None,None,None,Combination,"My mom, sister, and grandma all have dark circ...",P282938,Complete Eye Renewal Balm,6018,Algenist
4,1843,1843,P282938,sheri5,5,35-44,None,None,Fair,Normal,I had been using the Fresh black tea age delay...,P282938,Complete Eye Renewal Balm,6018,Algenist


## pie chart for 20 brands

In [329]:

def pie_chart_brand(data):
    '''
    pie chart for brands
    '''
    pie_chart=data.groupby('Brand_name').agg({'fileIndex':'count'})
    pie_chart.sort_values(by='fileIndex',inplace=True,ascending=False)
    pie_chart=pie_chart.iloc[:20]
    
    bupu = cl.scales['9']['seq']['BuPu']
    
    data_g = []
    
    tr_p = go.Pie(
        labels = pie_chart.index,
        values = pie_chart['fileIndex'],
        hoverinfo='label+percent+value', textinfo='label+percent',
        textfont=dict(size=15),
        marker=dict(colors= cl.interp( bupu, 20 ),
                    line=dict(color='#000000', width=1))
    )
    data_g.append(tr_p)
    layout = go.Layout(title="Top20 most popular brands",titlefont=dict(size=35))
    fig = go.Figure(data=data_g, layout=layout)
    plotly.offline.plot(fig,filename='brands')
    
pie_chart_brand(result_df)

/Users/xy/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `brands` didn't end with .html. Adding .html to the end of your file.



## pie chart for 20 products

In [330]:
def pie_chart_product(data):
    '''
    pie chart for products
    '''
    pie_chart=result_df.groupby(['Product_name','Brand_name']).agg({'fileIndex':'count'})
    pie_chart.reset_index(inplace=True)
    pie_chart.sort_values(by='fileIndex',inplace=True,ascending=False)
    pie_chart=pie_chart.iloc[:20]
    
    bupu = cl.scales['9']['seq']['BuPu']
    
    data_g = []
    tr_p = go.Pie(
        labels = pie_chart['Product_name'],
        values = pie_chart['fileIndex'],
        text=pie_chart['Brand_name'],
        hoverinfo='text+label+value+percent', textinfo='label+percent',
        textfont=dict(size=13),
        marker=dict(colors= cl.interp( bupu, 20 ),
                    line=dict(color='#000000', width=1))
    )
    data_g.append(tr_p)
    layout = go.Layout(title="Top20 most popular products",titlefont=dict(size=35))
    fig = go.Figure(data=data_g, layout=layout)
    plotly.offline.plot(fig,filename='products')
    
pie_chart_product(result_df)

/Users/xy/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `products` didn't end with .html. Adding .html to the end of your file.



## box plot of ratings for brands

In [331]:
def boxplot_rating_brand(data):
    data_g = []
    
    brand_rating=data.groupby('Brand_name').agg({'rating':np.median,'fileIndex':'count'})
    brand_rating.sort_values(by='rating',inplace=True,ascending=False)
    brand_rating=brand_rating[brand_rating['fileIndex']>5]  #select the brands having more than 5 reviews
     

    for col in brand_rating.index:
            data_g.append(go.Box(y=data[data['Brand_name'] == col]['rating'], name=col, showlegend=False))
    layout = go.Layout(title="Boxplot of ratings for each brand",titlefont=dict(size=35))
    fig = go.Figure(data=data_g, layout=layout)
            
    plotly.offline.plot(fig,filename='boxplot of ratings')
    
boxplot_rating_brand(result_df)

/Users/xy/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `boxplot of ratings` didn't end with .html. Adding .html to the end of your file.



## scatter plot for reviews

In [332]:
def scatter_review(data):
    colorscale = [[0, '#FAEE1C'], [0.33, '#F3558E'], [0.66, '#9C1DE7'], [1, '#581B98']]
    trace1 = go.Scatter(
    y = data['rating'],
    x = data['Brand_name'],
    mode='markers',
    text=data['review'],

    #textfont=dict(size=13),
    marker=dict(
        size='10',
        color = data['rating'], #set color equal to a variable
        colorscale=colorscale,
        showscale=True
        )
    )
    data_g= [trace1]
    plotly.offline.plot(data_g,filename='scatter plot for reviews')
    

scatter_review(result_df)

/Users/xy/anaconda3/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `scatter plot for reviews` didn't end with .html. Adding .html to the end of your file.



## trail for violin plot

In [334]:
brand_rating=result_df.groupby('Brand_name').agg({'fileIndex':'count'})
brand_rating=brand_rating[brand_rating['fileIndex']>20]  #select the brands having more than 15 reviews
brand_rating=result_df[result_df['Brand_name'].isin(brand_rating.index)]

group_stats = {}
groupby_data = brand_rating.groupby(['Brand_name'])



for group in brand_rating['Brand_name'].unique():
    data_from_group = groupby_data.get_group(group)['rating']
    stat = np.median(data_from_group)
    group_stats[group] = stat

fig = ff.create_violin(brand_rating, data_header='rating', group_header='Brand_name',
                       colors='YlOrRd', height=800, width=1300,
                        use_colorscale=True, group_stats=group_stats,title='Reviews for different brands')
plotly.offline.plot(fig)

'file:///Users/xy/Desktop/BIG DATA MGMT SYSTM/Project/temp-plot.html'